# Qwen2 新功能解析

在整体架构上，Qwen2 依旧沿用了与 Qwen1 一致的 **decoder-only** 架构。在模型架构层面，Qwen2 采用了标准的因果自注意力机制（Causal Self-Attention），并引入了 **GQA（Grouped Query Attention）** 替代传统的多头注意力（MHA），以优化 KV Cache 的内存占用，提升推理吞吐量。

在长上下文处理方面，Qwen2 采用了两项关键技术来实现超长序列的推理能力（最长支持 131,072 tokens）：

- **[DCA（Dual Chunk Attention）](https://arxiv.org/abs/2402.17463)**：这是一种 **training-free 的推理时优化技术**，不修改模型权重和架构定义，而是在推理时通过 monkey-patch 的方式替换标准注意力的计算逻辑。DCA 将长序列分成多个 chunk，通过三种不同的位置编码分别计算块内注意力、相邻块注意力和远程块注意力，再利用 log-sum-exp 技巧合并结果，从而将 RoPE 的位置映射回预训练时见过的范围内，解决了 RoPE 在超长序列上的外推失败问题。其实现位于独立项目 [ChunkLlama](https://github.com/HKUNLP/ChunkLlama) 中，而非 HuggingFace Transformers 的模型定义文件中。

- **[YaRN（Yet another RoPE extensioN）](https://arxiv.org/abs/2309.00071v2)**：用于对 RoPE 的注意力权重进行重新缩放，实现更好的长度外推能力。在 HuggingFace Transformers 中，YaRN 通过 `modeling_rope_utils.py` 中的 `ROPE_INIT_FUNCTIONS["yarn"]` 间接支持，需在模型配置中设置 `rope_type: "yarn"` 启用。GitHub 项目地址为 [jquesnelle/yarn](https://github.com/jquesnelle/yarn)。

在旋转位置编码（RoPE）方面，Qwen2 还将 RoPE 的 base frequency 从 10,000 提升至 1,000,000，以优化长上下文场景下的性能。

> **Qwen2 的长序列处理能力**：
>
> Qwen2 支持多种长度外推方案，以突破预训练长度的限制：
>
> - **YaRN 位置编码缩放**：通过对 RoPE 的逆频率进行分频段处理——高频维度保持外推、低频维度采用内插、中频维度平滑过渡——使模型在推理时能够处理远超预训练长度的序列。配合少量微调（约数百步），效果可进一步提升。
>
> - **分块注意力机制**（ChunkAttention）：通过将长序列切分为多个不超过预训练长度的分块，使每个块内的位置编码始终处于模型训练时见过的范围内。各块之间通过跨块注意力（搭配偏移位置编码）保留长距离依赖信息。该方案无需任何额外训练，直接替换注意力实现即可使用。
>
> 两种方案分别作用于不同层面——YaRN 修改位置编码的频率参数，分块注意力修改注意力的计算方式——因此可以独立使用，也可以组合使用以获得更好的效果。



In [ ]:
from typing import List, Optional, Tuple, Union

from torch import nn
import math
from transformers.models.llama.modeling_llama import rotate_half, repeat_kv
from transformers.modeling_outputs import CausalLMOutputWithPast
from torch.nn import CrossEntropyLoss
import torch
import transformers

from transformers.cache_utils import Cache
from flash_attn.flash_attn_interface import flash_attn_qkvpacked_func, flash_attn_func

# 基于分块计算的Qwen2实现，适用于预训练长度较长的场景
class ChunkLlamaRotaryEmbedding(nn.Module):
    def __init__(self, dim, max_position_embeddings=4096, base=10000, scaling_factor=1.0, device=None):
        super().__init__()

        self.max_seq_len = 16384  # 初始最大序列长度
        self.dim = dim  # RoPE编码的维度（通常是注意力头的维度head_size，如128）
        self.max_length = None  # 预训练时的最大序列长度（可选参数）
        self.scaling_factor = scaling_factor  # 位置编码的缩放因子，一般设置为1.0
        self.max_position_embeddings = max_position_embeddings  # 位置编码支持的最大长度
        self.base = base  # RoPE位置编码的基数，通常取值为10000
        
        # 预先计算并缓存RoPE的余弦和正弦值，初始缓存长度为max_seq_len
        self._set_cos_sin_cache(
            seq_len=self.max_seq_len,
            device=device, dtype=torch.float32
        )

    def _set_cos_sin_cache(self, seq_len, device, dtype):
        # 计算RoPE的逆频率
        inv_freq = 1.0 / (self.base ** (torch.arange(0, self.dim, 2).float().to(device) / self.dim))
        # RoPE的逆频率缓存不需要持久化存储，register_buffer表示该张量是模型的一部分，但不参与训练和保存
        self.register_buffer("inv_freq", inv_freq, persistent=False)

        # 局部窗口是用来保证相邻分块之间有重叠区域的，确保分块边界处的 token 也能看到足够的上下文
        chunk_len = chunk_size - local_window  # 分块的有效长度（总块长度 - 局部窗口长度）
        q_t = torch.arange(chunk_len, device=device, dtype=self.inv_freq.dtype) / self.scaling_factor  # 生成 Query 的块内位置 ID 序列
        # 生成 Query 用于跨块注意力的位置 ID序列，确保跨块位置编码与块内位置编码的连续性
        qc_t = (q_t + chunk_len).clamp(max=chunk_size) / self.scaling_factor
        # 生成 Key 的位置 ID 序列，长度为 seq_len + MAX_NEW_TOKENS，确保在推理阶段新生成的 token 也能获得正确的位置编码
        # 通过对位置 ID 取模 chunk_len，确保位置编码在每个分块内循环使用，适应不同长度的输入序列
        k_t = (torch.arange(seq_len + MAX_NEW_TOKENS, device=device,
                            dtype=self.inv_freq.dtype) % chunk_len) / self.scaling_factor

        q_freqs = torch.outer(q_t, self.inv_freq)  # 形状: [chunk_len, dim//2]
        qc_freqs = torch.outer(qc_t, self.inv_freq)  # 形状: [chunk_len, dim//2]
        k_freqs = torch.outer(k_t, self.inv_freq)  # 形状: [seq_len + MAX_NEW_TOKENS, dim//2]

        # 与论文实现略有不同，但通过不同的排列方式实现了相同的计算效果
        q_emb = torch.cat((q_freqs, q_freqs), dim=-1)  # 形状: [chunk_len, dim]
        qc_emb = torch.cat((qc_freqs, qc_freqs), dim=-1)  # 形状: [chunk_len, dim]
        k_emb = torch.cat((k_freqs, k_freqs), dim=-1)  # 形状: [seq_len + MAX_NEW_TOKENS, dim]
        
        # 注册缓存的余弦和正弦值张量，不持久化存储
        self.register_buffer("q_cos_cached", q_emb.cos().to(dtype), persistent=False)
        self.register_buffer("q_sin_cached", q_emb.sin().to(dtype), persistent=False)
        self.register_buffer("qc_cos_cached", qc_emb.cos().to(dtype), persistent=False)
        self.register_buffer("qc_sin_cached", qc_emb.sin().to(dtype), persistent=False)
        self.register_buffer("k_cos_cached", k_emb.cos().to(dtype), persistent=False)
        self.register_buffer("k_sin_cached", k_emb.sin().to(dtype), persistent=False)

    def forward(self, x, seq_len=None):
        # 输入x的形状: [批次大小bs, 注意力头数量num_attention_heads, 序列长度seq_len, 头维度head_size]
        # 确保所有token的位置都不超过分块大小chunk_size
        # 如果当前序列长度超过缓存的最大长度，重新计算并更新缓存
        if seq_len > self.max_seq_len:
            self._set_cos_sin_cache(seq_len=seq_len, device=self.inv_freq.device, dtype=torch.float32)
            self.max_seq_len = seq_len
        # 返回对应长度的余弦和正弦缓存值，并转换为输入x的数据类型
        return (
            self.q_cos_cached[:seq_len].to(dtype=x.dtype), 
            self.q_sin_cached[:seq_len].to(dtype=x.dtype),
            self.qc_cos_cached[:seq_len].to(dtype=x.dtype),
            self.qc_sin_cached[:seq_len].to(dtype=x.dtype),
            self.k_cos_cached[:seq_len].to(dtype=x.dtype),
            self.k_sin_cached[:seq_len].to(dtype=x.dtype),
        )

# 将分块注意力计算的结果进行合并
def merge_attn_outputs(flash_results):
    # 初始化注意力输出列表，包含第一个分块的结果
    attn_outputs_all = [flash_results[0][0]]
    flash_results = flash_results[1:]  # 移除第一个分块的结果，剩余的分块结果需要进行合并处理
    
    # 遍历剩余分块的计算结果
    for flash_per_chunk in flash_results:
        # 堆叠当前分块的所有注意力输出，形状为 [分块内注意力数, batch_size, num_heads, chunk_len, head_dim]
        attn_outputs = torch.stack([flash_attn_output[0] for flash_attn_output in flash_per_chunk])
        # 堆叠对应的logits值，形状为 [分块内注意力数, batch_size, num_heads, chunk_len]
        logits = torch.stack([flash_attn_output[1] for flash_attn_output in flash_per_chunk])
        # 计算logits的最大值用于数值稳定
        max_logits = torch.max(logits, dim=0).values  
        stable_logits = logits - max_logits.unsqueeze(0)
        # 计算softmax的指数部分（不带归一化）
        lse_s = torch.exp(stable_logits).detach()
        # 计算归一化因子
        lse_sum = torch.sum(lse_s, dim=0)
        lse_s /= lse_sum  # 维度为 [分块内注意力数, batch_size, num_heads, chunk_len]
        # 对注意力输出进行加权，得到当前分块的最终注意力输出，维度为[分块内注意力数, batch_size, num_heads, chunk_len, head_dim]
        attn_outputs *= lse_s.unsqueeze(-1)
        # 将加权后的结果求和并添加到列表中，维度为 [batch_size, num_heads, chunk_len, head_dim]
        attn_outputs_all.append(attn_outputs.sum(dim=0))
    
    # 拼接所有分块的注意力输出，得到完整序列的注意力输出，维度为 [batch_size, num_heads, seq_len, head_dim]
    return torch.cat(attn_outputs_all, dim=2)

# 执行flash attention计算的函数，适用于分块计算的场景
def do_flash_attn(query_states, key_states, value_states, causal=True):
    # 转换维度：将注意力头维度换到第二维以适配flash attention接口
    output, softmax_lse, _ = flash_attn_func(query_states.transpose(1, 2), key_states.transpose(1, 2),
                                             value_states.transpose(1, 2), causal=causal, return_attn_probs=True)
    # 转换回原始维度 [batch_size, num_heads, seq_len, head_dim] 
    # softmax_lse的形状为 [batch_size, num_heads, seq_len]，表示每个位置的softmax归一化因子
    return output.transpose(1, 2), softmax_lse


def apply_rotary_pos_emb(x, cos, sin, position_ids):
    # 应用旋转位置编码（RoPE）
    # cos和sin张量的前两个维度始终为1，可以压缩掉
    cos = cos.squeeze(1).squeeze(0)  # 形状: [序列长度seq_len, 维度dim]
    sin = sin.squeeze(1).squeeze(0)  # 形状: [序列长度seq_len, 维度dim]
    # 根据position_ids选取对应的位置编码，并添加注意力头维度
    cos = cos[position_ids].unsqueeze(1)  # 形状: [批次大小bs, 1, 序列长度seq_len, 维度dim]
    sin = sin[position_ids].unsqueeze(1)  # 形状: [批次大小bs, 1, 序列长度seq_len, 维度dim]
    # 计算旋转后的位置编码
    x_emb = (x * cos) + (rotate_half(x) * sin)
    return x_emb  # 形状为 [批次大小bs, 注意力头数量num_attention_heads, 序列长度seq_len, 维度dim]


def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_value: Optional[Cache] = None,
        output_attentions: bool = False,
        use_cache: bool = False,
        **kwargs,
) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
    # 获取输入的维度信息
    bsz, q_len, _ = hidden_states.size()
    chunk_len = chunk_size - local_window

    # 线性投影得到Q、K、V
    query_states = self.q_proj(hidden_states)
    key_states = self.k_proj(hidden_states)
    value_states = self.v_proj(hidden_states)

    # 重塑维度并转置：[batch_size, seq_len, num_heads * head_dim] -> [batch_size, num_heads, seq_len, head_dim]
    query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
    key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
    value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)

    # GQA: 扩展K、V的头数以匹配Q的头数（当使用分组注意力时）
    key_states = repeat_kv(key_states, self.num_key_value_groups)
    value_states = repeat_kv(value_states, self.num_key_value_groups)

    # 获取K/V的序列长度，长度为 seq_len （训练阶段）或 seq_len + 已缓存的长度（推理阶段）
    kv_seq_len = key_states.shape[-2]
    # 推理阶段（存在KV缓存）
    if past_key_value is not None:
        kv_seq_len += past_key_value.get_usable_length(kv_seq_len, self.layer_idx)

    # 获取Q的序列长度
    q_seq_len = query_states.shape[-2]
    # 判断是否使用了KV缓存（Q和KV序列长度不同），如果存在KV缓存，说明当前是推理阶段，需要进行分块计算
    has_kv_cache = q_seq_len != kv_seq_len
    
    # 获取旋转位置编码的余弦和正弦缓存
    q_cos, q_sin, qc_cos, qc_sin, k_cos, k_sin = self.rotary_emb(value_states, seq_len=kv_seq_len)
    # 对K应用旋转位置编码，之前 k_t 已经对position_ids取模chunk_len，确保位置编码在每个分块内循环使用
    key_states = apply_rotary_pos_emb(key_states, k_cos, k_sin, position_ids)
    # 对position_ids进行分块长度取模
    position_ids = position_ids % chunk_len

    # 更新KV缓存（如果存在）：将当前计算的 K、V 追加到缓存中，并返回完整的 K、V（包含历史）
    if past_key_value is not None:
        key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs=None)

    # 再次扩展KV头数（确保一致性）
    key_states = repeat_kv(key_states, self.num_key_value_groups)
    value_states = repeat_kv(value_states, self.num_key_value_groups)

    flash_results = []
    # 训练阶段（无KV缓存）
    if not has_kv_cache:
        # 处理第一个分块的Q（内部注意力）
        q_states_intra = apply_rotary_pos_emb(query_states[:, :, :chunk_len, :], q_cos, q_sin,
                                              position_ids[:, :chunk_len])
        k_states_prev = key_states[:, :, :chunk_len, :]
        v_states_prev = value_states[:, :, :chunk_len, :]
        # 计算第一个分块的flash attention
        flash_result = do_flash_attn(q_states_intra, k_states_prev, v_states_prev)
        flash_results.append(flash_result)
        # 计算剩余未处理的序列长度，长度为 seq_len - chunk_len（序列长度-分块有效长度）
        remain_len = kv_seq_len - chunk_len

        # 循环处理剩余分块，分块的注意力结果不止包含分块内部的注意力，还包含与前一个分块的注意力以及与更早分块的注意力
        while remain_len > 0:
            flash_per_chunk = []
            begin = kv_seq_len - remain_len  # 当前分块的起始位置
            curr_chunk_len = min(chunk_len, remain_len)  # 当前分块的实际长度（可能小于chunk_len）
            end = begin + curr_chunk_len  # 当前分块的结束位置

            # position_ids 将在每个分块内循环使用，确保位置编码都是从0开始，到chunk_len-1结束
            # 1. 注意力计算1：块内注意力
            q_states_intra = apply_rotary_pos_emb(query_states[:, :, begin:end, :], q_cos, q_sin,
                                                  position_ids[:, begin:end])
            k_states_intra = key_states[:, :, begin:end, :]  # 取当前分块的K
            v_states_intra = value_states[:, :, begin:end, :]  # 取当前分块的V
            flash_result = do_flash_attn(q_states_intra, k_states_intra, v_states_intra)
            flash_per_chunk.append(flash_result)

            # 2. 注意力计算2：与前一个分块的跨块注意力
            q_states_succ = apply_rotary_pos_emb(query_states[:, :, begin:end, :], qc_cos, qc_sin,
                                                 position_ids[:, begin:end])
            flash_result = do_flash_attn(q_states_succ, k_states_prev, v_states_prev, False)
            flash_per_chunk.append(flash_result)

            # 3. 注意力计算3：与更早分块的注意力
            if begin - (k_states_prev.size(-2)) > 0:
                prev_len = k_states_prev.size(-2)  # 前一个分块的长度
                # 对于更早的块，Q 与它们的距离已经非常远，精确的相对位置差异不太重要了，用固定的最大块内位置，简化计算同时近似效果足够好
                # q_states_inter 的维度为 [batch_size, num_heads, curr_chunk_len, head_dim]
                q_states_inter = apply_rotary_pos_emb(query_states[:, :, begin:end, :], qc_cos, qc_sin,
                                                      position_ids[:, chunk_len - 1][:, None].repeat(1, curr_chunk_len))
                k_states_inter = key_states[:, :, :begin - prev_len, :]
                v_states_inter = value_states[:, :, :begin - prev_len, :]
                flash_result = do_flash_attn(q_states_inter, k_states_inter, v_states_inter, False)
                flash_per_chunk.append(flash_result)

            # 添加当前分块的所有注意力结果
            flash_results.append(flash_per_chunk)
            # 更新前一个分块的KV引用
            k_states_prev = k_states_intra
            v_states_prev = v_states_intra
            # 减少剩余长度
            remain_len = remain_len - chunk_len

        # 合并所有分块的注意力输出，维度为 [batch_size, num_heads, seq_len, head_dim]
        attn_output = merge_attn_outputs(flash_results)
    # 推理阶段（有KV缓存）
    else:
        # 计算当前所在的分块编号
        chunk_num_curr = (kv_seq_len - 1) // chunk_len
        # 对Q应用旋转位置编码，维度为 [batch_size, num_heads, 1, head_dim]
        q_states_intra = apply_rotary_pos_emb(query_states, q_cos, q_sin, position_ids)
        # 获取当前分块的KV，维度为 [batch_size, num_heads, kv_seq_len - chunk_len * chunk_num_curr, head_dim]
        k_states_intra = key_states[:, :, chunk_len * chunk_num_curr:kv_seq_len, :]
        # 计算注意力权重，维度为 [batch_size, num_heads, 1, kv_seq_len - chunk_len * chunk_num_curr]
        attn_weights = torch.matmul(q_states_intra, k_states_intra.transpose(2, 3)) / math.sqrt(
            self.head_dim)
        attn_scores = [attn_weights]

        # 如果当前分块编号大于等于1，处理与前一个分块的注意力
        if chunk_num_curr >= 1:
            # 维度为 [batch_size, num_heads, 1, head_dim]
            q_states_succ = apply_rotary_pos_emb(query_states, qc_cos, qc_sin, position_ids)
            # 维度为 [batch_size, num_heads, chunk_len, head_dim]
            k_states_succ = key_states[:, :, chunk_len * (chunk_num_curr - 1):chunk_len * chunk_num_curr, :]
            # 维度为 [batch_size, num_heads, 1, chunk_len]
            attn_weights = torch.matmul(q_states_succ, k_states_succ.transpose(2, 3)) / math.sqrt(
                self.head_dim)
            attn_scores = [attn_weights] + attn_scores

        # 如果当前分块编号大于等于2，处理与更早分块的注意力
        if chunk_num_curr >= 2:
            # 对于更早的块，Q 与它们的距离已经非常远，精确的相对位置差异不太重要了，用固定的最大块内位置，简化计算同时近似效果足够好
            # 维度为 [batch_size, num_heads, 1, head_dim]
            q_states_inter = apply_rotary_pos_emb(query_states, qc_cos, qc_sin,
                                                  torch.tensor([[chunk_len - 1]], device=query_states.device))
            # 维度为 [batch_size, num_heads, chunk_len * (chunk_num_curr - 1), head_dim]
            k_states_inter = key_states[:, :, :chunk_len * (chunk_num_curr - 1), :]
            # 维度为 [batch_size, num_heads, 1, chunk_len * (chunk_num_curr - 1)]
            attn_weights = torch.matmul(q_states_inter, k_states_inter.transpose(2, 3)) / math.sqrt(
                self.head_dim)
            attn_scores = [attn_weights] + attn_scores

        # 拼接所有注意力分数，维度为 [batch_size, num_heads, 1, kv_seq_len]
        attn_weights = torch.cat(attn_scores, dim=-1)
        # 应用注意力掩码
        if attention_mask is not None:
            # 推理阶段时，q_len为1，kv_seq_len为当前分块的KV长度
            if attention_mask.size() != (bsz, 1, q_len, kv_seq_len):
                raise ValueError(
                    f"注意力掩码的形状应为 {(bsz, 1, q_len, kv_seq_len)}，但实际为 {attention_mask.size()}"
                )
            attn_weights = attn_weights + attention_mask

        # 将注意力权重提升到fp32计算softmax以提高数值稳定性
        attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query_states.dtype)
        # 计算注意力输出，维度为 [batch_size, num_heads, 1, head_dim]
        attn_output = torch.matmul(attn_weights, value_states)

    # 转置并整理注意力输出的维度
    attn_output = attn_output.transpose(1, 2).contiguous()
    attn_output = attn_output.reshape(bsz, q_len, self.hidden_size)
    # 通过输出投影层，维度为 [batch_size, seq_len, hidden_size]
    attn_output = self.o_proj(attn_output)
    return attn_output, None, past_key_value

def qwen_forward(
    self,
    input_ids: torch.LongTensor = None,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    past_key_values: Optional[List[torch.FloatTensor]] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    labels: Optional[torch.LongTensor] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
    *args, 
    **kwargs
) -> Union[Tuple, CausalLMOutputWithPast]:
    r"""
    参数:
        labels（形状为 `(batch_size, sequence_length)` 的 `torch.LongTensor` 类型，*可选*）：
            用于计算掩码语言建模损失（masked language modeling loss）的标签。标签索引值需满足以下任一条件：属于区间
            `[0, ..., config.vocab_size]`，或等于 -100（详见 `input_ids` 的文档字符串）。索引值设为 `-100` 的Token会被忽略
            （掩码处理），损失值仅针对索引落在 `[0, ..., config.vocab_size]` 范围内的Token计算。

    返回值：

    示例：

    ```python
    >>> from transformers import AutoTokenizer, Qwen2ForCausalLM

    >>> model = Qwen2ForCausalLM.from_pretrained(PATH_TO_CONVERTED_WEIGHTS)
    >>> tokenizer = AutoTokenizer.from_pretrained(PATH_TO_CONVERTED_TOKENIZER)

    >>> prompt = "嘿，你有自我意识吗？能和我聊聊吗？"
    >>> inputs = tokenizer(prompt, return_tensors="pt")

    >>> # 生成文本
    >>> generate_ids = model.generate(inputs.input_ids, max_length=30)
    >>> tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    "嘿，你有自我意识吗？能和我聊聊吗？
    我没有自我意识，但我可以和你交流。"
    """
    # 设置输出参数的默认值（优先使用传入值，否则使用模型配置）
    output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
    output_hidden_states = (
        output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
    )
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict

    # 调用模型主体的前向传播
    outputs = self.model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        position_ids=position_ids,
        past_key_values=past_key_values,
        inputs_embeds=inputs_embeds,
        use_cache=use_cache,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )

    # 获取最后一层的隐藏状态，维度为 [batch_size, seq_len, hidden_size]
    hidden_states = outputs[0]
    global full_logits_length

    # 处理短序列（长度小于完整logits长度）
    if hidden_states.shape[-2] < full_logits_length:
        # 通过语言模型头计算logits，维度为 [batch_size, seq_len, vocab_size]
        logits = self.lm_head(hidden_states)
        logits = logits.float()
        loss = None

        # 如果提供了标签，计算损失
        if labels is not None:
            # 移位操作：使第n个token预测第n+1个token
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # 展平token维度以计算损失
            loss_fct = CrossEntropyLoss()
            shift_logits = shift_logits.view(-1, self.config.vocab_size) # 维度为 [batch_size * (seq_len - 1), vocab_size]
            shift_labels = shift_labels.view(-1) # 维度为 [batch_size * (seq_len - 1)]
            # 确保标签和logits在同一设备上（支持模型并行）
            shift_labels = shift_labels.to(shift_logits.device)

            loss = loss_fct(shift_logits, shift_labels)
    # 处理长序列（分块计算）
    else:
        res = 0
        chunk_size = full_logits_length // 2
        if labels is None:  # 如果没有标签，直接计算最后一个token的logits用于推理
            logits = self.lm_head(hidden_states[..., -1:, :]) # 维度为 [batch_size, 1, vocab_size]
            logits = logits.float()
            loss = None
        else:
            # 分块计算损失
            shift_hidden_states = hidden_states[..., :-1, :] # 维度为 [batch_size, seq_len - 1, hidden_size]
            shift_labels = labels[..., 1:].contiguous() # 维度为 [batch_size, seq_len - 1]

            # 按分块遍历序列，shift_hidden_states.shape[-2] = seq_len - 1
            for i in range(0, shift_hidden_states.shape[-2], chunk_size):
                st = i
                ed = min(i + chunk_size, shift_hidden_states.shape[-2])
                # 对当前分块计算logits
                logits = self.lm_head(shift_hidden_states[..., st:ed, :])
                logits = logits.float()

                shift_logits = logits.contiguous()
                # 展平token维度
                loss_fct = CrossEntropyLoss()
                shift_logits = shift_logits.view(-1, self.config.vocab_size)
                shift_labels = shift_labels.view(-1)
                # 确保设备一致性
                shift_labels = shift_labels.to(shift_logits.device)
                
                # 累加加权损失（按分块长度加权）
                res = res + loss_fct(shift_logits, shift_labels[st:ed]) * (ed - st)
            # 计算平均损失
            loss = res / (hidden_states.shape[-2] - 1)
            logits = None

    # 非字典返回模式：返回元组形式的结果
    if not return_dict:
        output = (logits,) + outputs[1:]
        return (loss,) + output if loss is not None else output

    # 字典返回模式：返回结构化的输出对象
    return CausalLMOutputWithPast(
        loss=loss,
        logits=logits,
        past_key_values=outputs.past_key_values,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions,
    )

# 全局变量定义
chunk_size = None          # 分块大小
local_window = None        # 局部窗口大小
linear_factor = None       # 线性因子（预留）
MAX_NEW_TOKENS = 512       # 最大新生成token数


def replace_with_chunkqwen(pretraining_length=4096, local_window_size=None, full_logits_size=32000):
    """
    替换Qwen2模型的默认实现为分块计算版本
    
    参数:
        pretraining_length: 预训练序列长度（默认4096）
        local_window_size: 局部窗口大小（默认None，自动设为pretraining_length//16）
        full_logits_size: 完整logits计算的长度阈值（默认32000）
    """
    global chunk_size
    global local_window
    global full_logits_length
    # 计算分块大小（预训练长度的3/4）
    chunk_size = pretraining_length * 3 // 4
    # 设置局部窗口大小（用户指定或自动计算）
    local_window = local_window_size if local_window_size else pretraining_length // 16
    # 设置完整logits长度阈值
    full_logits_length = full_logits_size
    
    # 替换模型的核心组件实现
    transformers.models.qwen2.modeling_qwen2.Qwen2FlashAttention2.forward = forward
    transformers.models.qwen2.modeling_qwen2.Qwen2RotaryEmbedding = ChunkLlamaRotaryEmbedding
    transformers.models.qwen2.modeling_qwen2.Qwen2ForCausalLM.forward = qwen_forward

## YaRN 解析
### 核心问题解析
#### 第一部分：为什么直接外推（Direct Extrapolation）不行？
**—— 罪魁祸首是“低频项”的越界（OOD），而不是高频项。**

在 RoPE 中，位置编码是由不同频率的正弦/余弦函数组成的向量。
*   **高频维度（High Frequency）：** 比如向量的前几维，波长很短，转得飞快。
*   **低频维度（Low Frequency）：** 比如向量的后几维，波长很长，转得很慢。

**论文观点（Section 3.2）：**
1.  **低频项记录的是“绝对位置”：** 在预训练长度 $L$ 内，低频维度的旋转角度可能连一圈都没转完（比如只转了 $0 \to \pi/4$）。模型记住了这个范围。
2.  **直接外推导致分布外（OOD）：** 如果你把长度拉长到 $2L$，低频维度的角度突然变成了 $\pi/2$。这是模型在训练时从未见过的数值。模型会不知所措，导致注意力机制崩溃。

**结论：** 直接外推失效，是因为**低频项**跑到了模型没见过的区域。


#### 第二部分：为什么简单的线性插值（PI）效果一般？
**—— 因为它“误伤”了高频项，导致模型“近视”。**

既然直接外推会导致低频越界，Chen et al. (2023) 提出的 PI 方法就是把所有维度都“压扁”（插值），把 $0 \to 2L$ 压回 $0 \to L$。

**论文观点（Section 3.1）：**
1.  **高频项即“分辨率”：** RoPE 的高频项类似于傅里叶特征（Fourier Features）。高频分量的作用是让模型能够分辨**紧邻的两个 token**（比如位置 100 和 101）。
2.  **插值导致高频丢失：** 如果你把高频项也压扁（除以 Scale $s$），相邻两个 token 之间的旋转角度差就会变小。
    *   原来：100 和 101 差了 30度（清晰可辨）。
    *   插值后：100 和 101 只差了 15度（变得模糊）。
3.  **后果：** 模型的“视力”下降，分不清谁前谁后，导致微调后短文本的困惑度（Perplexity）反而上升。

**结论：** 简单的插值解决了低频越界问题，但因为压缩了高频项，破坏了局部位置的分辨率。


#### 第三部分：YaRN 的核心策略（NTK-by-parts）
**—— 分而治之：该压缩的压缩，该保留的保留。**

YaRN（以及 NTK-aware）的精髓在于：**不同频率的维度，承载的信息类型不同，处理方式也必须不同。**

论文根据波长 $\lambda$ 和上下文长度 $L$ 的关系，把维度分成了三类（Section 3.2）：

##### 1. 高频维度（$\lambda \ll L$）
*   **特征：** 波长远小于上下文长度。这些维度在训练时已经转了成千上万圈。
*   **模型学到了什么？** **旋转不变性（Rotational Invariance）**。模型只关心相对角度差，不关心绝对角度是多少。
*   **YaRN 的做法：** **完全不插值（Do not interpolate）**。
    *   既然模型只看相对关系，那外推时继续转圈就行了，不用担心越界。
    *   **目的：** 保持原汁原味的旋转速度，**保留最高的分辨率**，让模型在长文本里依然能看清相邻词的顺序。

##### 2. 低频维度（$\lambda \ge L$）
*   **特征：** 波长大于或等于上下文长度。这些维度转得很慢。
*   **模型学到了什么？** **绝对位置信息**。模型记住了具体的角度数值范围。
*   **YaRN 的做法：** **必须插值（Interpolate）**。
    *   如果不插值，角度就会跑出训练范围（OOD）。必须把它压回去。
    *   **目的：** 消除分布外问题，让长文本的全局位置看起来像短文本。

##### 3. 中频维度
*   **做法：** 在上述两者之间做一个平滑过渡（Ramp function），避免突变。


#### 第四部分：被忽视的细节 —— 注意力熵（Entropy）
**—— 为什么还需要乘以 $\sqrt{t}$（Temperature）？**

除了位置编码的频率问题，论文在 **Section 3.4** 还提到了一个关键点：**注意力分布的尖锐程度。**

*   **现象：** 当你把上下文窗口拉长（比如 4k $\to$ 128k），Token 的数量变多了，点积（Dot Product）的数量级和分布可能会发生变化。这会导致 Softmax 算出来的概率分布变得**过于平滑**（关注不到重点）或者**过于尖锐**（只关注一两个词）。
*   **YaRN 的做法：** 引入一个温度系数 $t$（通常是 $\sqrt{t}$ 缩放），用来修正 logits 的分布。
    *   公式：$\text{softmax}(\frac{q^T k}{t\sqrt{|D|}})$
*   **目的：** 这不是为了解决位置编码的“混乱”，而是为了让模型在处理长序列时，注意力的**熵（Entropy）**保持在训练时的水平，防止注意力机制因为候选词太多而“失焦”。


#### 总结：论文的完整逻辑链

1.  **直接外推（Extrapolation）失败** $\leftarrow$ **低频项**角度越界，模型没见过（OOD）。
2.  **简单插值（PI）失败** $\leftarrow$ **高频项**被压缩，局部差异变小，模型变“瞎”（分辨率丢失）。
3.  **YaRN/NTK 成功** $\leftarrow$ **混合策略**：
    *   **不动高频项：** 保护视力（分辨率）。
    *   **压缩低频项：** 防止越界（OOD）。
    *   **调整温度 $t$：** 维持注意力的集中度（Entropy）。


---

### YaRN 公式解析

#### 1. 判定基准：波长与上下文长度的比率 $r$

首先，RoPE 的每个维度 $d$ 都有一个对应的旋转频率 $\theta_d$。频率越高，波长越短。
YaRN 定义了一个关键指标：**波长 $\lambda_d$**（Wavelength）。

$$
\lambda_d = \frac{2\pi}{\theta_d} = 2\pi b^{\frac{2d}{|D|}}
$$

*   $b=10000$（基数）。
*   $|D|$ 是隐藏层维度。
*   **物理意义：** $\lambda_d$ 表示在这个维度上，位置编码旋转一整圈（$2\pi$）需要多少个 token。

然后，定义比率 $r$：**预训练上下文长度 $L$ 与波长 $\lambda_d$ 的比值。**

$$
r(d) = \frac{L}{\lambda_d}
$$

*   **如果 $r(d)$ 很小（$< 1$）：** 说明 $\lambda_d > L$。波长比训练长度还长。这意味着在训练期间，这个维度连一圈都没转完。模型学到的是**绝对位置**。
    *   **策略：** 必须插值（Interpolate），防止溢出（OOD）。
*   **如果 $r(d)$ 很大：** 说明 $\lambda_d \ll L$。波长很短，在这个长度内已经转了无数圈。模型学到的是**相对位置**。
    *   **策略：** 保持原样（Extrapolate），保留高频分辨率。

#### 2. 混合策略：Ramp Function $\gamma(r)$

YaRN 并不是非黑即白地切分，而是引入了一个 **Ramp 函数（斜坡函数）** $\gamma$ 来做平滑过渡。它定义了两个阈值 $\alpha$ 和 $\beta$（论文中通常取 $\alpha=1, \beta=32$）。

$$
\gamma(r) = 
\begin{cases} 
0, & \text{if } r < \alpha \quad (\text{低频，只插值}) \\
1, & \text{if } r > \beta \quad (\text{高频，只外推}) \\
\frac{r - \alpha}{\beta - \alpha}, & \text{otherwise} \quad (\text{中频，混合})
\end{cases}
$$

*   **$\gamma=0$ (低频部分)：** 完全使用线性插值（PI），把频率除以扩展倍数 $s$。
*   **$\gamma=1$ (高频部分)：** 完全保持原始频率，不做任何缩放。
*   **$0 < \gamma < 1$ (中频部分)：** 在两者之间线性混合。

**最终的频率修正公式：**

$$
h(\theta_d) = (1 - \gamma(r)) \underbrace{\frac{\theta_d}{s}}_{\text{PI插值}} + \gamma(r) \underbrace{\theta_d}_{\text{原始频率}}
$$

*   **解读这个公式：**
    *   对于低频项（$r$ 很小，$\gamma \approx 0$），公式变成了 $\frac{\theta_d}{s}$。这就是标准的线性插值（PI）。把频率变慢，让位置编码“缩”回来，适应长文本。
    *   对于高频项（$r$ 很大，$\gamma \approx 1$），公式变成了 $\theta_d$。这就是不做任何改变，保留高频的分辨率。
    *   这就是 **"NTK-by-parts"** 的核心：**分频段处理**。


#### 3. 注意力修正：Temperature Scaling $\sqrt{t}$

除了位置编码的频率调整，YaRN 发现当上下文窗口扩大（比如从 4k 到 128k）时，Attention Logits 的分布会发生变化（熵增）。为了对抗这种分布变化，YaRN 引入了一个温度系数 $t$。

**注意力公式修正为：**

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{Q K^T}{t \sqrt{|D|}}\right) V
$$

*   注意这里分母多了一个 $t$。
*   为了实现简单（兼容 Flash Attention），我们不需要改 Softmax 的代码，只需要在输入 RoPE 之前，把 $Q$ 和 $K$ 向量乘以一个缩放因子：

$$
\text{Scale Factor} = \sqrt{\frac{1}{t}}
$$

**温度 $t$ 的计算公式（论文 Eq. 22）：**

$$
\sqrt{\frac{1}{t}} = 0.1 \ln(s) + 1
$$

*   $s$ 是扩展倍数（比如 4k 扩到 128k，$s=32$）。
*   **原理解析：**
    *   当 $s=1$（不扩展）时，$\ln(1)=0$，缩放因子为 1，无影响。
    *   当 $s > 1$（扩展）时，缩放因子 $> 1$。这意味着 $\frac{1}{t} > 1$，即 $t < 1$（温度降低）。
    *   **降低温度的作用：** 使得 Softmax 分布更**尖锐（Sharper）**。
    *   因为长文本中 Token 太多了，注意力容易被稀释（变得太均匀），降低温度有助于让模型更集中地关注那些真正重要的 Token。


#### 4. 总结 YaRN 的完整流程

当你想要把模型从长度 $L$ 扩展到 $s \times L$ 时，YaRN 对每个维度 $d$ 执行以下操作：

1.  **算波长：** 计算该维度的波长 $\lambda_d$。
2.  **算比率：** 计算 $r = L / \lambda_d$。
3.  **算权重：** 根据 $r$ 和阈值 $\alpha, \beta$ 计算混合权重 $\gamma$。
4.  **改频率：** 用公式 $h(\theta_d) = (1-\gamma)\frac{\theta_d}{s} + \gamma\theta_d$ 计算新的旋转频率。
    *   低频项变慢了（被 $s$ 除）。
    *   高频项没变。
5.  **改幅度（Scale）：** 把 Embedding 乘以 $\sqrt{\frac{1}{t}} = 0.1\ln(s)+1$，让注意力分布变尖锐。

这就是 YaRN 能够兼顾**全局一致性（低频不越界）**和**局部敏锐度（高频不丢失）**的数学原理。

---

In [ ]:
def _compute_yarn_parameters(
    config: "PreTrainedConfig",
    device: Optional["torch.device"] = None,
    seq_len: int | None = None,
    layer_type: str | None = None,
) -> tuple["torch.Tensor", float]:
    """
    基于NTK缩放计算逆频率。相关原理可参考
    [原始论文](https://huggingface.co/papers/2309.00071)

    参数:
        config ([`~transformers.PreTrainedConfig`]):
            模型配置类。该函数假定配置对象至少会提供以下属性：

            *   rope_theta (`float`): 用于推导逆频率的基准波长。
            *   hidden_size (`int`): 若未直接提供head_dim，则作为推导head_dim时的分子。
            *   num_attention_heads (`int`): 若未直接提供head_dim，则作为推导head_dim时的分母。
            *   max_position_embeddings (`int`): 位置嵌入的最大长度。
            *   rope_parameters (`dict[str, float | int]`): 标准的RoPE缩放参数，会从中读取以下键值：
                *   `attention_factor` (`float`, *可选*): 应用于计算得到的余弦/正弦值的缩放因子。
                    若为None，该值会根据可用的`factor`、`mscale`和`mscale_all_dim`推导得出。
                *   `beta_fast` (`float`, *可选*，默认值为32): 用于设置线性斜坡函数中仅外推（extrapolation）部分的边界参数。
                *   `beta_slow` (`float`, *可选*，默认值为1): 用于设置线性斜坡函数中仅内插（interpolation）部分的边界参数。
                *   `factor` (`float`, *可选*): 对内插位置ID以扩展上下文长度时应用的缩放因子。此外，若`attention_factor`为None，
                    该值的对数会用于计算`attention_factor`的值（若提供了`mscale`和`mscale_all_dim`，可能会结合这两个参数一起计算）。
                *   `mscale` (`float`, *可选*): 若`attention_factor`为None且同时提供了`mscale`和`mscale_all_dim`，
                    在推导`attention_factor`的分子时，`mscale`会作为标量对`log(factor)`进行修正。若未提供，
                    `attention_factor`仅基于`factor`计算。
                *   `mscale_all_dim` (`float`, *可选*): 若`attention_factor`为None且同时提供了`mscale`和`mscale_all_dim`，
                    在推导`attention_factor`的分母时，`mscale_all_dim`会作为标量对`log(factor)`进行修正。若未提供，
                    `attention_factor`仅基于`factor`计算。
                *   `original_max_position_embeddings` (`int`): 预训练阶段使用的原始最大位置嵌入长度。
                *   `truncate` (`bool`, *可选*): 是否截断修正范围。

            此外，若配置对象中包含以下属性，该函数也会加以利用：

            *   head_dim (`int`, *可选*): 模型中键值头（key-value heads）的维度。若为None，该值会通过
                hidden_size // num_attention_heads 推导得出。
            *   partial_rotary_factor (`float`, *可选*，默认值为1.0): 若该值小于1.0，则仅返回head_dim前一部分维度对应的逆频率。
        device (`torch.device`):
            用于初始化逆频率张量的设备。
        seq_len (`int`, *可选*):
            当前序列长度。该参数对此类RoPE不生效。

    返回值:
        一个包含两个元素的元组 (`torch.Tensor`, `float`)，其中第一个元素是RoPE嵌入的逆频率张量，
        第二个元素是应用于余弦/正弦计算结果的后处理缩放因子。
    """
    # 为了向后兼容，如果 `rope_parameters_dict` 使用旧格式则进行标准化处理
    config.standardize_rope_params()
    # 根据是否指定layer_type，获取对应层的RoPE参数或全局RoPE参数
    rope_parameters_dict = config.rope_parameters[layer_type] if layer_type is not None else config.rope_parameters

    # 提取基础参数
    base = rope_parameters_dict["rope_theta"]  # RoPE基准波长参数
    # 部分旋转因子（控制仅对部分维度应用RoPE），默认1.0（全部维度）
    partial_rotary_factor = rope_parameters_dict.get("partial_rotary_factor", 1.0)
    # 计算实际参与旋转的维度：优先使用配置中的head_dim，否则用hidden_size/注意力头数推导
    head_dim = getattr(config, "head_dim", config.hidden_size // config.num_attention_heads)
    dim = int(head_dim * partial_rotary_factor)  # 最终参与RoPE的维度

    # 提取缩放相关参数
    factor = rope_parameters_dict["factor"]  # 位置ID内插缩放因子
    attention_factor = rope_parameters_dict.get("attention_factor")  # 注意力缩放因子
    mscale = rope_parameters_dict.get("mscale")  # mscale修正系数
    mscale_all_dim = rope_parameters_dict.get("mscale_all_dim")  # 全维度mscale修正系数
    # 预训练阶段的原始最大位置嵌入长度
    original_max_position_embeddings = rope_parameters_dict["original_max_position_embeddings"]

    # 注意：DeekSeek-V3（以及其他潜在模型）的`original_max_position_embeddings`字段
    # 存储了预训练时的取值。它们使用`max_position_embeddings`与该值的比值
    # 来计算默认的注意力缩放因子，而非直接使用`factor`。
    if factor is None:
        factor = config.max_position_embeddings / original_max_position_embeddings

    def get_mscale(scale, mscale=1):
        """
        根据缩放因子计算mscale修正后的缩放值
        参数:
            scale: 基础缩放因子
            mscale: 修正系数，默认1
        返回:
            修正后的缩放值（scale<=1时返回1.0，否则按公式计算）
        """
        if scale <= 1:
            return 1.0
        return 0.1 * mscale * math.log(scale) + 1.0

    # 按照论文建议设置注意力因子
    if attention_factor is None:
        # 若同时提供mscale和mscale_all_dim，用两者的比值计算
        if mscale and mscale_all_dim:
            attention_factor = float(get_mscale(factor, mscale) / get_mscale(factor, mscale_all_dim))
        else:
            # 否则仅基于factor计算
            attention_factor = get_mscale(factor)

    # 可选配置项
    # beta_fast/beta_slow: 论文建议的默认值分别为32/1
    beta_fast = rope_parameters_dict.get("beta_fast") or 32
    beta_slow = rope_parameters_dict.get("beta_slow") or 1

    # 计算逆频率
    def find_correction_dim(num_rotations, dim, base, max_position_embeddings):
        """根据旋转次数反向推导对应的维度（逆维度公式）
        参数:
            num_rotations: 旋转次数
            dim: 总维度
            base: RoPE基准波长（rope_theta）
            max_position_embeddings: 最大位置嵌入长度
        返回:
            对应旋转次数的维度值
        """
        return (dim * math.log(max_position_embeddings / (num_rotations * 2 * math.pi))) / (2 * math.log(base))

    def find_correction_range(low_rot, high_rot, dim, base, max_position_embeddings, truncate):
        """根据旋转次数范围找到对应的维度范围边界
        参数:
            low_rot: 最小旋转次数
            high_rot: 最大旋转次数
            dim: 总维度
            base: RoPE基准波长
            max_position_embeddings: 最大位置嵌入长度
            truncate: 是否截断为整数
        返回:
            修正后的维度范围（low, high），确保在0~dim-1范围内
        """
        # low对应于仅外推的边界，high对应于仅内插的边界
        # low_rot = beta_fast，high_rot = beta_slow
        # 0~low: 主要外推区域，low~high: 内插与外推过渡区域，high~dim: 主要内插区域
        low = find_correction_dim(low_rot, dim, base, max_position_embeddings)
        high = find_correction_dim(high_rot, dim, base, max_position_embeddings)
        if truncate:
            low = math.floor(low)  # 向下取整
            high = math.ceil(high)  # 向上取整
        return max(low, 0), min(high, dim - 1)  # 确保范围有效

    def linear_ramp_factor(min, max, dim):
        """生成线性斜坡函数因子（用于平滑过渡内插/外推区域）
        参数:
            min: 斜坡起始维度
            max: 斜坡结束维度
            dim: 总维度
        返回:
            形状为(dim,)的张量，值在0~1之间线性变化
        """
        if min == max:
            max += 0.001  # 防止分母为0（奇点）

        # 计算线性函数：(当前维度 - 起始维度) / (结束维度 - 起始维度)
        # 0 表示完全外推，1 表示完全内插，介于两者之间表示过渡区域
        linear_func = (torch.arange(dim, dtype=torch.float32) - min) / (max - min)
        ramp_func = torch.clamp(linear_func, 0, 1)  # 限制在0~1范围内
        return ramp_func

    # 变量命名说明："interpolation"（内插）来自原始技术，指通过缩放位置ID来扩展上下文长度
    # 换句话说，内插 = 应用缩放因子
    # 1. 计算基础位置频率：base^((0,2,4...dim-2)/dim)
    pos_freqs = base ** (torch.arange(0, dim, 2).to(device=device, dtype=torch.float) / dim)
    # 2. 外推模式逆频率：不缩放的原始逆频率（用于超出预训练长度的位置）
    inv_freq_extrapolation = 1.0 / pos_freqs
    # 3. 内插模式逆频率：缩放后的逆频率（用于预训练长度内的位置）
    inv_freq_interpolation = 1.0 / (factor * pos_freqs)

    # 是否截断修正范围（默认True）
    truncate = config.rope_parameters.get("truncate", True)
    # 计算内插/外推的维度边界（基于beta_fast/beta_slow）
    low, high = find_correction_range(beta_fast, beta_slow, dim, base, original_max_position_embeddings, truncate)

    # 获取用于外推的n维旋转缩放修正因子，因为 linear_ramp_factor 结果为 0 表示完全外推
    # 注意：dim//2是因为RoPE仅对偶数维度计算（每两个维度为一组）
    inv_freq_extrapolation_factor = 1 - linear_ramp_factor(low, high, dim // 2).to(device=device, dtype=torch.float)
    # 融合内插和外推的逆频率：
    # - 内插部分：inv_freq_interpolation * (1 - 外推因子)
    # - 外推部分：inv_freq_extrapolation * 外推因子
    # 最终实现不同维度上内插/外推的平滑过渡
    inv_freq = (
        inv_freq_interpolation * (1 - inv_freq_extrapolation_factor)
        + inv_freq_extrapolation * inv_freq_extrapolation_factor
    )
    return inv_freq, attention_factor

## 训练后阶段
在大规模预训练完成后，我们对 Qwen2 进行了训练后优化。这一阶段对提升模型在多领域的能力至关重要，包括代码编写、数学计算、逻辑推理、指令遵循及多语言理解等。同时，训练后阶段也确保模型生成内容符合人类价值观，做到有用、诚实且无害。

与传统严重依赖大量人工监督的方案不同，本方法注重以**最少人工标注**实现可扩展的对齐。具体而言，我们通过多种方式构建高质量演示数据与偏好数据，用于监督微调（SFT）和基于人类反馈的强化学习（RLHF），在最小化人工标注成本的同时，保证数据质量与可靠性。

### 训练后数据
训练后数据主要分为两类：
- **演示数据** $D = \{(x_i, y_i)\}$
- **偏好数据** $P = \{(x_i, y_i^+, y_i^-)\}$

其中：
- $x_i$ 表示用户指令
- $y_i$ 表示符合要求的标准回应
- $y_i^+$ 和 $y_i^-$ 为同一指令下的一对回应，且 $y_i^+$ 质量优于 $y_i^-$

演示数据 $D$ 用于监督微调（SFT），偏好数据 $P$ 用于基于人类反馈的强化学习（RLHF）。

整体训练数据构建分为两步：**协同数据标注**与**自动化数据合成**。
1. 从大规模指令语料库中提取数据本体，得到广泛且多样的高质量指令集，并通过系统优化提升指令复杂度；
2. 通过人工标注得到目标回应 $y_i$ 与优劣回应对 $(y_i^+, y_i^-)$，再结合多种自动化对齐策略，在代码、数学、指令遵循、创作、角色扮演及安全等领域合成大量高质量数据。

#### 协同数据标注
- **自动本体提取**  
  首先使用 InsTag ——一种开放集细粒度标注工具，从大规模指令数据集中提取底层本体，再通过人工优化保证本体准确性。

- **指令筛选**  
  对带标注标签的每条指令，从标签多样性、语义丰富度、复杂度及意图完整性等维度进行评估，筛选出高代表性指令集。

- **指令演化**  
  采用自演化策略，利用 Qwen 模型为现有指令增加约束与要求，提升指令复杂度，确保数据集覆盖不同难度等级。

- **人工标注**  
  通过多种生成策略与不同规模的 Qwen 模型，为每条指令生成多个候选回应。标注人员对回应进行质量排序，确保最优回应满足规范，最终形成可用的演示数据与偏好数据。

#### 自动化数据合成
在大规模场景下，依靠纯人工维持标注质量成本高、难度大，尤其在需要专业知识、细致度与耐心的任务上更为明显。为此，我们设计了多种自动化对齐策略，实现数据规模化合成：

- **拒绝采样（Rejection Sampling）**  
  针对数学等答案明确的任务，使用拒绝采样提升解的质量。让大模型为每条指令生成多条推理路径，仅保留答案正确、逻辑合理的路径作为演示数据；通过正确与错误路径对比，构建偏好数据。

- **执行反馈（Execution Feedback）**  
  面向代码任务，利用大模型生成解决方案与测试用例，通过编译与执行评估有效性，自动生成演示数据与偏好数据。该方法也可用于指令遵循评估：对带长度限制等约束的指令，由模型生成 Python 验证函数，确保输出符合要求。

- **数据再利用（Data Repurposing）**  
  针对文学创作等对标注人员要求较高的场景，收集公共领域高质量文本，由大模型生成不同粒度的指令，并与原文配对作为演示数据。
  例如，从维基百科等知识库抽取角色档案，指导模型生成对应的指令与回应，构建高质量角色扮演数据，保证角色设定完整一致。

- **宪法反馈（Constitutional Feedback）**  
  基于宪法人工智能思想，预先定义安全与价值观相关准则，构建宪法数据集。利用该数据集引导模型生成符合或偏离准则的回应，作为演示数据与偏好数据的重要来源，确保模型输出安全、合规。

### 监督微调（Supervised Fine-Tuning, SFT）  
我们构建了包含**50 万+示例**的大规模指令数据集，覆盖指令遵循、代码、数学、推理、创作、角色扮演、多语言与安全等能力。

训练细节：
- 序列长度：32768 tokens
- 训练轮次：2 个 epoch
- 学习率：从 $7 \times 10^{-6}$ 线性衰减至 $7 \times 10^{-7}$
- 权重衰减：0.1
- 梯度裁剪：最大范数 1.0

通过以上配置，在提升指令遵循能力的同时缓解过拟合。

### 基于人类反馈的强化学习（RLHF）
Qwen2 的 RLHF 流程分为**离线训练**与**在线训练**两个连续阶段：

1. **离线训练**  
   使用预先构建好的偏好数据集 $P$，通过 **直接偏好优化（DPO）** 最大化优质回应 $y_i^+$ 与劣质回应 $y_i^-$ 之间的似然差异。

2. **在线训练**  
   模型利用奖励模型的实时反馈进行迭代优化：从当前策略模型采样多条回应，由奖励模型选出最优与最劣样本，形成新的偏好对，用于每一轮 DPO 训练。

此外，为缓解**对齐损耗（alignment tax）**——即模型在对齐人类偏好时出现的通用能力下降问题，我们采用了**在线融合优化器（Online Merging Optimizer）**。